### Listing the required libraries

In [ ]:
import sys
!pip install pandas
!pip install numpy
!pip install datapane
!pip install plotly

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import datapane as dp

### Connection to the database

In [ ]:
#put the name of your database here:
db = 'MusicStore.sqlite'


def run_query(q):   #
    with sqlite3.connect(db) as conn:
        return pd.read_sql(q,conn)
    
def run_command(c):
    with sqlite3.connect(db) as conn:
        conn.isolation_level = None
        conn.execute(c) 


### Testing the connection by printing the name and number of records for all the tables available in the database

In [ ]:
def show_tables():
    q = '''
        SELECT
            name
        FROM sqlite_master
        WHERE type IN ("table","view");
        '''
    return run_query(q)

def get_table_row_count(tablename):
    q = '''
        SELECT
            COUNT(1)
        FROM %s;
        ''' % tablename
    return run_query(q)["COUNT(1)"][0]

tables = show_tables()
tables["row_count"] = [get_table_row_count(t) for t in tables["name"]]

tables

### Sample code that shows how to print the columns in each table

In [ ]:
conn = sqlite3.connect(db)
cursor = conn.cursor()


print("Columns of Table InvoiceLine")
data=cursor.execute('''SELECT * FROM InvoiceLine''')
for column in data.description:
    print(column[0])
print("-------------------------------")
    
    
print("Columns of Table Album")
data=cursor.execute('''SELECT * FROM Album''')
for column in data.description:
    print(column[0])
print("-------------------------------")


print("Columns of Table Customer")
data=cursor.execute('''SELECT * FROM Customer''')
for column in data.description:
    print(column[0])       
print("-------------------------------")

### Copy your Datapane token (from the website) to create the online dashboard

In [ ]:
# copy your datapane token below
!datapane login --token=

### How to run a query

In [ ]:
#After you develop your query in the sqlite developer (oracle developer), you can copy it here and 
#give it a name (such as top_10_tracks_query) 

top_10_tracks_query = '''
SELECT 
    t.Name trackname,
    a.Title album_title,
    ar.Name artist,
    COUNT(*) as total_purchases,
    SUM(il.UnitPrice) total_cost
FROM track t 
JOIN album a on a.AlbumId = t.AlbumId
JOIN artist ar on ar.ArtistId = a.ArtistId
JOIN InvoiceLine il on il.TrackId = t.TrackId
GROUP BY 1
ORDER BY total_purchases desc
LIMIT 10
'''

# use run_query to run the it and see the results
top_10_df = run_query(top_10_tracks_query)

top_10_df

# dp.Report(
#     dp.DataTable(top_10_df)
# ).upload(name="Music Sales")

In [ ]:
employee_sales_performance = '''
WITH 
    customer_support_rep_sales AS
        (
         SELECT 
             i.CustomerId,
             strftime('%Y-%m', i.InvoiceDate) || "-01" Month,
             c.SupportRepId,
             SUM(i.Total) Sales_Total
         FROM invoice i
         INNER JOIN customer c ON c.CustomerId = i.CustomerId
         GROUP BY 2, 3
        )
SELECT
    e.FirstName || " " || e.LastName "Employee Name",
    csrs.Month,
    SUM(csrs.Sales_total) "Amount of Sales (in $)"
FROM customer_support_rep_sales csrs
INNER JOIN employee e ON csrs.SupportRepId = e.EmployeeId
GROUP BY 1, 2;
'''

run_query(employee_sales_performance)

In [ ]:
employee_sales = run_query(employee_sales_performance)
employee_sales["Month"] = pd.to_datetime(employee_sales["Month"])

employee_sales_monthly = px.line(employee_sales,
                            x = "Month",
                            y = "Amount of Sales (in $)",
                            color = "Employee Name")

employee_sales_monthly.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

employee_sales_monthly.show()

In [ ]:
employee_sales_total = employee_sales.groupby(["Employee Name"]) \
                                        .sum() \
                                        .sort_values(by="Amount of Sales (in $)") \
                                        .reset_index()

employee_sales_bar = px.bar(employee_sales_total, 
                            y = 'Employee Name', 
                            x = "Amount of Sales (in $)",
                            text = "Amount of Sales (in $)",
                            orientation = 'h')

employee_sales_bar.update_traces(texttemplate='%{text:.1f}')
employee_sales_bar.show()

In [ ]:
albums_to_purchase = '''
WITH 
    usa_tracks_sold AS
        (
         SELECT il.* FROM InvoiceLine il
         INNER JOIN Invoice i ON il.InvoiceId = i.InvoiceId
         INNER JOIN Customer c ON i.CustomerId = c.CustomerId
        )
SELECT 
    g.name Genre, 
    COUNT(uts.InvoiceLineId) "Number of Tracks Sold", 
    CAST(COUNT(uts.InvoiceLineId) AS FLOAT)/(SELECT COUNT(*) FROM usa_tracks_sold) "Percentage of Tracks Sold"
FROM usa_tracks_sold uts
INNER JOIN track t ON uts.TrackId = t.TrackId
INNER JOIN genre g ON t.GenreId = g.GenreId
GROUP BY 1 
ORDER BY 2 DESC
LIMIT 10;
'''

genre_sales = run_query(albums_to_purchase)
genre_sales.sort_values('Number of Tracks Sold', inplace=True)

genre_sales_plot = px.bar(
    genre_sales, 
    x="Percentage of Tracks Sold", 
    y="Genre", 
    orientation='h',
    text="Percentage of Tracks Sold",
    title = "Sales by Genre"
)

genre_sales_plot.update_traces(texttemplate='%{text:.1%}', textposition='outside')

genre_sales_plot.show()

In [ ]:
   
sales_by_country = '''
WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customerid,
       il.*
     FROM invoiceline il
     INNER JOIN invoice i ON i.invoiceid = il.invoiceid
     INNER JOIN customer c ON c.customerid = i.customerid
    )


SELECT
    country,
    count(distinct customerid) customers,
    SUM(unitprice) totalsales,
    SUM(unitprice) / count(distinct customerid) customerlifetimevalue,
    SUM(unitprice) / count(distinct invoiceid) averageorder,
    CASE
        WHEN country = "Other" THEN 1
        ELSE 0
    END AS sort
FROM country_or_other
GROUP BY country
ORDER BY sort ASC, totalsales DESC;
'''

run_query(sales_by_country)

In [ ]:
country_metrics = run_query(sales_by_country)

sales_breakdown = go.Figure(data=[go.Pie(
    labels = country_metrics["country"], 
    values=country_metrics["totalsales"], 
    hole=.3)
                                 ])

sales_breakdown.update_layout(title_text="Sales by country")

sales_breakdown.show()

In [ ]:
customer_ratio = country_metrics["customers"] / country_metrics["customers"].sum() * 100.0
sales_ratio = country_metrics["totalsales"] / country_metrics["totalsales"].sum() * 100.0


custs_vs_dollars = go.Figure(data=[
    go.Bar(name='Customers', x=country_metrics["country"], y=customer_ratio),
    go.Bar(name='Total Sales', x=country_metrics["country"], y=sales_ratio),
])

# Change the bar mode
custs_vs_dollars.update_layout(barmode='group', title_text="Pct customers vs sales")
custs_vs_dollars.show()

In [ ]:
country_metrics["differencefromavg"] = (country_metrics["averageorder"] - \
                                          country_metrics["averageorder"].mean()) * 100.0

avg_order = px.bar(
    country_metrics, 
    x = "country", 
    y = "differencefromavg", 
    color = "country", 
    title = "Avg order - Pct difference from mean",
    labels={
        "country": "Country",
        "differencefromavg": "Difference from mean (%)"
    }
)

avg_order.update_layout(showlegend=False)

avg_order.show()

In [ ]:
clv = px.bar(
    country_metrics,
    x = 'country',
    y = 'customerlifetimevalue',
    color='country',
    labels={
        "country": "Country",
        "customerlifetimevalue": "Customer Lifetime Value ($)"
    },
    title = "Customer Lifetime Value"
)

clv.update_layout(showlegend=False)


clv.show()

In [ ]:
customers = tables[tables["name"]=="Customer"]["row_count"].values[0]
employees = tables[tables["name"]=="Employee"]["row_count"].values[0]
tracks_sold = tables[tables["name"]=="InvoiceLine"]["row_count"].values[0]

In [ ]:
r = dp.Report( 
    dp.Group(
        dp.BigNumber(heading="Total customers", value = customers),
        dp.BigNumber(heading="Total employees", value = employees),
        dp.BigNumber(heading="Total tracks sold", value = tracks_sold),
        columns = 3,
        name="Little_group"
    ),
    dp.Plot(genre_sales_plot, name = "genre_sales"),
    dp.Plot(employee_sales_monthly, name = "employee-sales-1"),
    dp.Plot(employee_sales_bar, name = "employee-sales-2"),
    dp.Group(
        dp.Plot(sales_breakdown),
        dp.Plot(custs_vs_dollars),
        dp.Plot(avg_order),
        dp.Plot(clv),
        columns = 2
    ),
    dp.DataTable(top_10_df, name = "track-sales"),
).upload(name="Interactive Dashboard using SQL")